In [3]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

import pandas as pd 
import ast
import numpy as np
import json
from tqdm import tqdm
from IPython.display import clear_output
import os

from src.Retriever import ThresholdRetriever
from src.Reader import LLM_Model
from src.Scorer import SimilarityScorerConfig
from src.utils import ReaderMetrics, RetrieverMetrics, save_rag_trial_log, prepare_thresholdretriever_configs, prepare_reader_configs, load_benchmarks_df
from src.utils import evaluate_retriever, evaluate_reader

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SyntaxError: invalid syntax (evaluate_parts.py, line 62)

In [2]:
# !!! TO CHANGE !!!
TRIAL = 3
BENCHMARKS_MAXSIZE = 500
BENCHMARKS_INFO = {'mtssquad': {'db': 'v2', 'table': 'v1'}}

# Retriever part
RETRIEVER_PARAMS = {
    "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
    "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
    "model_kwargs": {'device':'cuda'},
    "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
    "params": {'fetch_k': 50, 'threshold': 0.23, 'max_k': 3}
}

# Reader part
READER_PARAMS = {
    'prompts': {
        "assistant": 'Отвечай на вопросы, используя информацию из текстов в списке ниже. Если ты не уверена в релевантности данных текстов по отношению к заданному вопросу, то сгенерируй следующий ответ: "У меня нет ответа на ваш вопрос.".',
        "system": "Ты вопросно-ответная система. Все ответы генерируй на русском языке. По вопросам отвечай кратко, чётко и конкретно. Не генерируй излишнюю информацию.",
    },
    'gen': {'max_new_tokens': 512, 'eos_token_id': 79097},
    'data_operate': {'batch_size': 1}
    }

ADDITIONAL_PARAMS = {
    'topk_score_list': 3
}

BERTSCORE_MODEL_PATH = "ru_electra_medium"

# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_READERCACHE = f'{SAVE_LOGDIR}/reader_cache.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

##### Configure Retriever-part

In [3]:
retrievers_config, benchmarks_path = prepare_thresholdretriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
retriever_metrics = RetrieverMetrics()

In [4]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

##### Configure Reader-part

In [6]:
reader_config = prepare_reader_configs(READER_PARAMS)
READER = LLM_Model(reader_config)

Loading checkpoint shards: 100%|██████████| 4/4 [03:21<00:00, 50.45s/it]


In [10]:
sim_score_config = SimilarityScorerConfig()
reader_metrics = ReaderMetrics(BASE_DIR, BERTSCORE_MODEL_PATH, sim_score_config, READER)

Loading Meteor...
Loading ExactMatch


In [12]:
RETRIEVERS = {name: ThresholdRetriever(config) for name, config in retrievers_config.items()}

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/rag_project/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


##### Evaluating pipeline

In [14]:
retriever_scores, retriever_cache_ids, predicted_chunks, cache_relevant_flags = evaluate_retriever(benchmarks_df, RETRIEVERS, retriever_metrics,
                                                                            show_step=5, topk_score_list=ADDITIONAL_PARAMS['topk_score_list'])

mtssquad


100%|██████████| 500/500 [00:08<00:00, 61.27it/s, MRR=0.778, mAP=0.778, Recall=0.839, Precision=0.28, F1=0.419, NoRelContextScore=nan] 


In [15]:
contexts = {name: [reader_config.prompts.assistant + "\n\n" + "\n\n".join([f'{i+1}. {doc[1]}' for i, doc in enumerate(docs)]) 
                                                                      for docs in chunks] for name, chunks in predicted_chunks.items()}

In [18]:
reader_scores, reader_cache = evaluate_reader(benchmarks_df, READER, reader_metrics, contexts, show_step=1, cache_relevant_flags=cache_relevant_flags)

0it [00:00, ?it/s]

False
Нет, Мадонна не просила изменить аранжировку песни.
Да, по просьбе Мадонны заменили аранжировщика Каминса на более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса.


1it [00:04,  4.50s/it, BLEU2=nan, BLEU1=nan, ExactMatch=nan, METEOR=nan, BertScore=nan, StubScore=0]

[0]
False
Первый сингл Мадонны был "Everybody", выпущенный в 1982 году на лейбле Sire.
Песню Мадонны "Everybody" выпускают в качестве сингла.


2it [00:08,  4.23s/it, BLEU2=nan, BLEU1=nan, ExactMatch=nan, METEOR=nan, BertScore=nan, StubScore=0]

...

True
Александр Грин был сослан в Пинегу, Архангельскую губернию, после третьего ареста.
После третьего ареста писатель Александр Грин был сослан в Пинегу Архангельской губернии.
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


499it [25:28,  2.07s/it, BLEU2=0.343, BLEU1=0.416, ExactMatch=0.031, METEOR=0.601, BertScore=nan, StubScore=0.263] 

True
Нансен обнаружил, что айсберги могут переносить минеральный материал, который может дать представление о геологическом строении неисследованных областей Гренландии.
О том, что айсберги могут переносить значительные количества минерального материала, дающего представление о геологическом строении неисследованных областей Гренландии, обнаружил Фритьоф Нансен.
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


500it [25:29,  3.06s/it, BLEU2=0.343, BLEU1=0.416, ExactMatch=0.031, METEOR=0.6, BertScore=nan, StubScore=0.263]  

True
27 июня Викинг оказался заперт в сплошных льдах и начался дрейф.
27 июня Викинг Нансена оказался затёрт сплошными ледовыми полями, начался незапланированный дрейф.
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


In [19]:
save_rag_trial_log(SAVE_LOGDIR, reader_scores, retriever_scores, 
                   SAVE_HYPERPARAMS, SAVE_READERCACHE, SAVE_RETRIEVERCACHE,
                   reader_cache, retriever_cache_ids, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE,
                   READER_PARAMS, RETRIEVER_PARAMS, ADDITIONAL_PARAMS)